# 🔬 NAT (Neighborhood Attention Transformer) - BreaKHis Dataset

Bu notebook, BreaKHis meme kanseri histopatoloji veri seti üzerinde NAT modelini eğitir.

**Özellikler:**
- Tüm magnification'lar (40X, 100X, 200X, 400X)
- Patient-stratified split
- Confusion Matrix, ROC-AUC, Precision, Recall, F1-Score
- GPU optimizasyonları (A100/T4/V100)

---
**⚠️ ÖNEMLİ:** Runtime > Change runtime type > GPU (A100 veya T4) seçin!


In [ ]:
# =============================================================================
# 1️⃣ GPU Kontrolü ve Kütüphaneler
# =============================================================================

!nvidia-smi

import torch
print(f"\n✅ PyTorch: {torch.__version__}")
print(f"✅ CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Gerekli kütüphaneleri yükle
%pip install -q kagglehub tqdm seaborn scikit-learn


In [ ]:
# =============================================================================
# 2️⃣ Veri Setini İndir
# =============================================================================

import kagglehub

print("📥 Downloading BreaKHis dataset...")
dataset_path = kagglehub.dataset_download("ambarish/breakhis")
print(f"✅ Dataset downloaded to: {dataset_path}")


In [ ]:
# =============================================================================
# 3️⃣ Imports ve Konfigürasyon
# =============================================================================

import os
import random
import re
from pathlib import Path
from typing import Tuple, Dict, List, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.amp import GradScaler, autocast
import torchvision.transforms as T
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    precision_recall_fscore_support,
    roc_auc_score,
    roc_curve
)

# Seed
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Device
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Device: {DEVICE}")


In [ ]:
# =============================================================================
# 4️⃣ Konfigürasyon - Colab/A100 için Optimize
# =============================================================================

class Config:
    # Veri seti yolu
    BASE_PATH = Path(dataset_path) / "BreaKHis_v1" / "BreaKHis_v1" / "histology_slides" / "breast"
    
    # Eğitim parametreleri
    SEED = 42
    IMG_SIZE = 224
    # A100: 64, T4/V100: 32, düşük VRAM: 16
    BATCH_SIZE = 64 if torch.cuda.is_available() and torch.cuda.get_device_properties(0).total_memory > 30e9 else 32
    NUM_WORKERS = 2  # Colab için
    EPOCHS = 20  # Başlangıç için yeterli
    LEARNING_RATE = 1e-4
    WEIGHT_DECAY = 0.05
    
    # Model
    NUM_CLASSES = 2
    CLASS_NAMES = ['benign', 'malignant']
    MODEL_VARIANT = 'tiny'
    
    # Magnification - Tümü
    MAGNIFICATIONS = ['40X', '100X', '200X', '400X']
    SELECTED_MAG = 'ALL'
    
    # Early stopping
    PATIENCE = 10
    USE_AMP = True
    
    DEVICE = DEVICE

config = Config()
print(f"📊 Batch Size: {config.BATCH_SIZE}")
print(f"📊 Epochs: {config.EPOCHS}")
print(f"📊 Image Size: {config.IMG_SIZE}")


In [ ]:
# =============================================================================
# 5️⃣ NAT (Neighborhood Attention Transformer) Model
# =============================================================================

class NeighborhoodAttention2D(nn.Module):
    """Efficient 2D Neighborhood Attention"""
    
    def __init__(self, dim, num_heads, kernel_size=7, dilation=1, qkv_bias=True, attn_drop=0.0, proj_drop=0.0):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        self.scale = self.head_dim ** -0.5
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.padding = (kernel_size // 2) * dilation
        
        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.rpb = nn.Parameter(torch.zeros(num_heads, kernel_size * kernel_size))
        nn.init.trunc_normal_(self.rpb, std=0.02)
        
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)
        
    def forward(self, x):
        B, H, W, C = x.shape
        
        qkv = self.qkv(x).reshape(B, H, W, 3, self.num_heads, self.head_dim)
        qkv = qkv.permute(3, 0, 4, 5, 1, 2)
        q, k, v = qkv[0], qkv[1], qkv[2]
        
        q = q * self.scale
        
        k = F.pad(k, (self.padding,) * 4, mode='constant', value=0)
        v = F.pad(v, (self.padding,) * 4, mode='constant', value=0)
        
        k = k.unfold(3, self.kernel_size, 1).unfold(4, self.kernel_size, 1)
        v = v.unfold(3, self.kernel_size, 1).unfold(4, self.kernel_size, 1)
        
        k = k.reshape(B, self.num_heads, self.head_dim, H, W, -1)
        v = v.reshape(B, self.num_heads, self.head_dim, H, W, -1)
        
        q = q.permute(0, 1, 3, 4, 2).unsqueeze(-2)
        k = k.permute(0, 1, 3, 4, 2, 5)
        v = v.permute(0, 1, 3, 4, 2, 5)
        
        attn = q @ k
        attn = attn + self.rpb.view(1, self.num_heads, 1, 1, 1, -1)
        attn = F.softmax(attn, dim=-1)
        attn = self.attn_drop(attn)
        
        out = attn @ v.transpose(-2, -1)
        out = out.squeeze(-2).permute(0, 2, 3, 1, 4).reshape(B, H, W, C)
        
        out = self.proj(out)
        out = self.proj_drop(out)
        
        return out


class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, drop=0.0):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class DropPath(nn.Module):
    def __init__(self, drop_prob=0.0):
        super().__init__()
        self.drop_prob = drop_prob
        
    def forward(self, x):
        if self.drop_prob == 0.0 or not self.training:
            return x
        keep_prob = 1 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
        random_tensor.floor_()
        return x.div(keep_prob) * random_tensor


class NATBlock(nn.Module):
    def __init__(self, dim, num_heads, kernel_size=7, dilation=1, mlp_ratio=4.0, 
                 qkv_bias=True, drop=0.0, attn_drop=0.0, drop_path=0.0):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = NeighborhoodAttention2D(dim, num_heads, kernel_size, dilation, qkv_bias, attn_drop, drop)
        self.drop_path = DropPath(drop_path) if drop_path > 0.0 else nn.Identity()
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = Mlp(dim, int(dim * mlp_ratio), drop=drop)
        
    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x


class PatchEmbed(nn.Module):
    def __init__(self, in_chans=3, embed_dim=64, patch_size=4):
        super().__init__()
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.norm = nn.LayerNorm(embed_dim)
        
    def forward(self, x):
        x = self.proj(x)
        x = x.permute(0, 2, 3, 1)
        x = self.norm(x)
        return x


class PatchMerging(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.reduction = nn.Linear(4 * dim, 2 * dim, bias=False)
        self.norm = nn.LayerNorm(4 * dim)
        
    def forward(self, x):
        B, H, W, C = x.shape
        if H % 2 == 1 or W % 2 == 1:
            x = F.pad(x, (0, 0, 0, W % 2, 0, H % 2))
            B, H, W, C = x.shape
        x0 = x[:, 0::2, 0::2, :]
        x1 = x[:, 1::2, 0::2, :]
        x2 = x[:, 0::2, 1::2, :]
        x3 = x[:, 1::2, 1::2, :]
        x = torch.cat([x0, x1, x2, x3], dim=-1)
        x = self.norm(x)
        x = self.reduction(x)
        return x


class NATStage(nn.Module):
    def __init__(self, dim, depth, num_heads, kernel_size=7, dilations=None, 
                 downsample=True, mlp_ratio=4.0, qkv_bias=True, drop=0.0, attn_drop=0.0, drop_path=None):
        super().__init__()
        dilations = dilations or [1] * depth
        drop_path = drop_path or [0.0] * depth
        
        self.blocks = nn.ModuleList([
            NATBlock(dim, num_heads, kernel_size, dilations[i], mlp_ratio, qkv_bias, drop, attn_drop, drop_path[i])
            for i in range(depth)
        ])
        self.downsample = PatchMerging(dim) if downsample else None
        
    def forward(self, x):
        for blk in self.blocks:
            x = blk(x)
        if self.downsample is not None:
            x = self.downsample(x)
        return x


class NAT(nn.Module):
    def __init__(self, img_size=224, patch_size=4, in_chans=3, num_classes=2,
                 embed_dim=64, depths=[2, 2, 6, 2], num_heads=[2, 4, 8, 16],
                 kernel_size=7, mlp_ratio=4.0, qkv_bias=True,
                 drop_rate=0.0, attn_drop_rate=0.0, drop_path_rate=0.2):
        super().__init__()
        
        self.num_classes = num_classes
        self.num_stages = len(depths)
        self.num_features = embed_dim * (2 ** (self.num_stages - 1))
        
        self.patch_embed = PatchEmbed(in_chans, embed_dim, patch_size)
        
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]
        
        self.stages = nn.ModuleList()
        for i in range(self.num_stages):
            stage = NATStage(
                dim=embed_dim * (2 ** i),
                depth=depths[i],
                num_heads=num_heads[i],
                kernel_size=kernel_size,
                downsample=(i < self.num_stages - 1),
                mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias,
                drop=drop_rate,
                attn_drop=attn_drop_rate,
                drop_path=dpr[sum(depths[:i]):sum(depths[:i+1])],
            )
            self.stages.append(stage)
        
        self.norm = nn.LayerNorm(self.num_features)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.head = nn.Linear(self.num_features, num_classes)
        
        self.apply(self._init_weights)
        
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=0.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
                
    def forward(self, x):
        x = self.patch_embed(x)
        for stage in self.stages:
            x = stage(x)
        x = self.norm(x)
        x = x.permute(0, 3, 1, 2).flatten(2)
        x = self.avgpool(x).flatten(1)
        x = self.head(x)
        return x


def nat_tiny(num_classes=2):
    return NAT(embed_dim=64, depths=[3, 4, 6, 5], num_heads=[2, 4, 8, 16], num_classes=num_classes)

def nat_mini(num_classes=2):
    return NAT(embed_dim=64, depths=[2, 2, 6, 2], num_heads=[2, 4, 8, 16], num_classes=num_classes)

print("✅ NAT Model defined!")


In [ ]:
# =============================================================================
# 6️⃣ Veri Seti Hazırlama
# =============================================================================

def parse_breakhis_path(filepath):
    fname = os.path.basename(filepath)
    parts = fname.split('_')
    label_token = parts[1] if len(parts) > 1 else ''
    label = 'benign' if label_token.upper().startswith('B') else 'malignant'
    mag = Path(filepath).parents[0].name
    try:
        patient = parts[2].rsplit('-', 2)[0]
    except:
        m = re.search(r'([A-Za-z]-\d+-\w+)', fname)
        patient = m.group(1) if m else fname
    return label, mag, patient


def create_dataframe(base_path):
    image_paths = sorted([str(p) for p in Path(base_path).rglob('*.png')])
    rows = []
    for p in image_paths:
        label, mag, patient = parse_breakhis_path(p)
        rows.append({'filepath': p, 'label': label, 'mag': mag.upper().replace(' ', ''), 'patient_id': patient})
    return pd.DataFrame(rows)


def patient_stratified_split(df, train_frac=0.7, val_frac=0.1, test_frac=0.2, seed=42):
    patients = df['patient_id'].unique().tolist()
    random.Random(seed).shuffle(patients)
    n = len(patients)
    n_train = int(round(train_frac * n))
    n_val = int(round(val_frac * n))
    
    train_patients = set(patients[:n_train])
    val_patients = set(patients[n_train:n_train + n_val])
    test_patients = set(patients[n_train + n_val:])
    
    return {
        'train': df[df['patient_id'].isin(train_patients)].reset_index(drop=True),
        'val': df[df['patient_id'].isin(val_patients)].reset_index(drop=True),
        'test': df[df['patient_id'].isin(test_patients)].reset_index(drop=True)
    }


# DataFrame oluştur
df = create_dataframe(config.BASE_PATH)
print(f"📊 Total images: {len(df)}")

# Magnification dağılımı
print(f"\n📊 Magnification distribution:")
for mag in config.MAGNIFICATIONS:
    count = len(df[df['mag'] == mag])
    print(f"   - {mag}: {count}")

# Class dağılımı
print(f"\n📊 Class distribution:")
for cls in config.CLASS_NAMES:
    count = len(df[df['label'] == cls])
    print(f"   - {cls}: {count} ({100*count/len(df):.1f}%)")

# Patient-stratified split
splits = patient_stratified_split(df, 0.7, 0.1, 0.2, config.SEED)
print(f"\n📊 Splits:")
for name, split_df in splits.items():
    print(f"   - {name}: {len(split_df)} images, {split_df['patient_id'].nunique()} patients")


In [ ]:
# =============================================================================
# 7️⃣ Dataset & DataLoader
# =============================================================================

class BreaKHisDataset(Dataset):
    def __init__(self, df, transform=None, class_names=['benign', 'malignant']):
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.class_to_idx = {c: i for i, c in enumerate(class_names)}
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['filepath']).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.class_to_idx[row['label']]
        return image, label


def get_transforms(img_size=224, is_training=True):
    if is_training:
        return T.Compose([
            T.Resize((img_size + 32, img_size + 32)),
            T.RandomCrop(img_size),
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.RandomRotation(15),
            T.ColorJitter(0.2, 0.2, 0.1, 0.05),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
    else:
        return T.Compose([
            T.Resize((img_size, img_size)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])


# DataLoader'lar
train_ds = BreaKHisDataset(splits['train'], get_transforms(config.IMG_SIZE, True), config.CLASS_NAMES)
val_ds = BreaKHisDataset(splits['val'], get_transforms(config.IMG_SIZE, False), config.CLASS_NAMES)
test_ds = BreaKHisDataset(splits['test'], get_transforms(config.IMG_SIZE, False), config.CLASS_NAMES)

train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_ds, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=config.NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=config.NUM_WORKERS, pin_memory=True)

# Class weights
class_counts = splits['train']['label'].value_counts()
total = len(splits['train'])
class_weights = torch.tensor([total / (2 * class_counts.get(cls, 1)) for cls in config.CLASS_NAMES], dtype=torch.float32)
class_weights = class_weights.to(config.DEVICE)

print(f"✅ DataLoaders created!")
print(f"   Train batches: {len(train_loader)}, Val batches: {len(val_loader)}, Test batches: {len(test_loader)}")
print(f"   Class weights: {class_weights.cpu().numpy()}")


In [ ]:
# =============================================================================
# 8️⃣ Model Oluştur ve Eğit
# =============================================================================

# Model
model = nat_tiny(num_classes=config.NUM_CLASSES).to(config.DEVICE)
total_params = sum(p.numel() for p in model.parameters())
print(f"🏗️ NAT Model - Total params: {total_params:,}")

# Loss, Optimizer, Scheduler
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.EPOCHS, eta_min=1e-6)
scaler = GradScaler('cuda') if config.USE_AMP else None

# History
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'val_f1': [], 'val_auc': []}
best_val_f1 = 0.0
patience_counter = 0

print("🚀 Starting training...")
print("=" * 70)

for epoch in range(config.EPOCHS):
    # TRAIN
    model.train()
    train_loss, train_correct, train_total = 0.0, 0, 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.EPOCHS} [Train]")
    for images, labels in pbar:
        images, labels = images.to(config.DEVICE), labels.to(config.DEVICE)
        optimizer.zero_grad()
        
        if config.USE_AMP:
            with autocast('cuda'):
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        train_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()
        pbar.set_postfix({'loss': f'{loss.item():.4f}', 'acc': f'{100.*train_correct/train_total:.2f}%'})
    
    train_loss /= train_total
    train_acc = train_correct / train_total
    
    # VALIDATE
    model.eval()
    val_loss, val_preds, val_labels, val_probs = 0.0, [], [], []
    
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{config.EPOCHS} [Val]"):
            images, labels = images.to(config.DEVICE), labels.to(config.DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)
            probs = torch.softmax(outputs, dim=1)
            
            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            val_preds.extend(predicted.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
            val_probs.extend(probs[:, 1].cpu().numpy())
    
    val_loss /= len(val_labels)
    val_acc = accuracy_score(val_labels, val_preds)
    _, _, val_f1, _ = precision_recall_fscore_support(val_labels, val_preds, average='weighted', zero_division=0)
    try:
        val_auc = roc_auc_score(val_labels, val_probs)
    except:
        val_auc = 0.0
    
    scheduler.step()
    
    # History
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_f1'].append(val_f1)
    history['val_auc'].append(val_auc)
    
    print(f"\n📊 Epoch {epoch+1}: Train Loss={train_loss:.4f}, Acc={100*train_acc:.2f}% | Val Loss={val_loss:.4f}, Acc={100*val_acc:.2f}%, F1={100*val_f1:.2f}%, AUC={100*val_auc:.2f}%")
    
    # Best model
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        patience_counter = 0
        torch.save(model.state_dict(), 'nat_best_model.pth')
        print(f"   ✅ Best model saved! (F1: {100*best_val_f1:.2f}%)")
    else:
        patience_counter += 1
        if patience_counter >= config.PATIENCE:
            print(f"\n⚠️ Early stopping at epoch {epoch+1}")
            break

print("\n" + "=" * 70)
print(f"✅ Training complete! Best Val F1: {100*best_val_f1:.2f}%")


In [ ]:
# =============================================================================
# 9️⃣ Eğitim Grafikleri
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].plot(history['train_loss'], label='Train', lw=2)
axes[0, 0].plot(history['val_loss'], label='Val', lw=2)
axes[0, 0].set_title('Loss', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(history['train_acc'], label='Train', lw=2)
axes[0, 1].plot(history['val_acc'], label='Val', lw=2)
axes[0, 1].set_title('Accuracy', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(history['val_f1'], label='Val F1', lw=2, color='green')
axes[1, 0].set_title('Validation F1-Score', fontsize=14, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(history['val_auc'], label='Val AUC', lw=2, color='purple')
axes[1, 1].set_title('Validation AUC-ROC', fontsize=14, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=150)
plt.show()


In [ ]:
# =============================================================================
# 🔟 Test Değerlendirmesi
# =============================================================================

# En iyi modeli yükle
model.load_state_dict(torch.load('nat_best_model.pth'))
model.eval()

test_preds, test_labels_list, test_probs = [], [], []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing"):
        images = images.to(config.DEVICE)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)
        _, predicted = outputs.max(1)
        
        test_preds.extend(predicted.cpu().numpy())
        test_labels_list.extend(labels.numpy())
        test_probs.extend(probs.cpu().numpy())

test_preds = np.array(test_preds)
test_labels_arr = np.array(test_labels_list)
test_probs = np.array(test_probs)

# Metrikler
accuracy = accuracy_score(test_labels_arr, test_preds)
precision, recall, f1, _ = precision_recall_fscore_support(test_labels_arr, test_preds, average='weighted')
auc = roc_auc_score(test_labels_arr, test_probs[:, 1])

print("\n" + "=" * 70)
print(" TEST RESULTS ")
print("=" * 70)
print(f"   Accuracy:  {100*accuracy:.2f}%")
print(f"   Precision: {100*precision:.2f}%")
print(f"   Recall:    {100*recall:.2f}%")
print(f"   F1-Score:  {100*f1:.2f}%")
print(f"   AUC-ROC:   {100*auc:.2f}%")
print("=" * 70)

print("\n📋 Classification Report:")
print(classification_report(test_labels_arr, test_preds, target_names=config.CLASS_NAMES, digits=4))


In [ ]:
# =============================================================================
# 1️⃣1️⃣ Confusion Matrix & ROC Curve
# =============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion Matrix
cm = confusion_matrix(test_labels_arr, test_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=config.CLASS_NAMES, yticklabels=config.CLASS_NAMES,
            annot_kws={'size': 16, 'weight': 'bold'})
axes[0].set_xlabel('Predicted', fontsize=12, fontweight='bold')
axes[0].set_ylabel('True', fontsize=12, fontweight='bold')
axes[0].set_title('Confusion Matrix', fontsize=14, fontweight='bold')

# ROC Curve
fpr, tpr, _ = roc_curve(test_labels_arr, test_probs[:, 1])
axes[1].plot(fpr, tpr, color='#3498db', lw=2, label=f'ROC (AUC = {auc:.4f})')
axes[1].plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
axes[1].fill_between(fpr, tpr, alpha=0.3)
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('False Positive Rate', fontsize=12, fontweight='bold')
axes[1].set_ylabel('True Positive Rate', fontsize=12, fontweight='bold')
axes[1].set_title('ROC Curve', fontsize=14, fontweight='bold')
axes[1].legend(loc='lower right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('confusion_roc.png', dpi=150)
plt.show()


In [ ]:
# =============================================================================
# 1️⃣2️⃣ Per-Magnification Sonuçları
# =============================================================================

print("\n📊 Per-Magnification Results:")
print("-" * 70)
print(f"{'Mag':<10} {'Accuracy':<12} {'Precision':<12} {'Recall':<12} {'F1':<12} {'Support':<10}")
print("-" * 70)

results_per_mag = {}

for mag in config.MAGNIFICATIONS:
    df_mag = splits['test'][splits['test']['mag'] == mag]
    if len(df_mag) == 0:
        continue
    
    ds = BreaKHisDataset(df_mag, get_transforms(config.IMG_SIZE, False), config.CLASS_NAMES)
    loader = DataLoader(ds, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=2)
    
    preds, labels = [], []
    with torch.no_grad():
        for images, lbls in loader:
            outputs = model(images.to(config.DEVICE))
            _, predicted = outputs.max(1)
            preds.extend(predicted.cpu().numpy())
            labels.extend(lbls.numpy())
    
    acc = accuracy_score(labels, preds)
    prec, rec, f1_sc, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    
    results_per_mag[mag] = {'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1_sc, 'support': len(df_mag)}
    print(f"{mag:<10} {100*acc:<12.2f} {100*prec:<12.2f} {100*rec:<12.2f} {100*f1_sc:<12.2f} {len(df_mag):<10}")

print("-" * 70)

# Bar Chart
mags = list(results_per_mag.keys())
metrics = ['accuracy', 'precision', 'recall', 'f1']
colors = ['#3498db', '#2ecc71', '#f39c12', '#e74c3c']

fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(mags))
width = 0.2

for i, (metric, color) in enumerate(zip(metrics, colors)):
    values = [results_per_mag[mag][metric] for mag in mags]
    ax.bar(x + i * width, values, width, label=metric.capitalize(), color=color)

ax.set_ylabel('Score', fontsize=12)
ax.set_title('Performance by Magnification', fontsize=14, fontweight='bold')
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels(mags)
ax.legend()
ax.set_ylim(0, 1.1)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('per_magnification.png', dpi=150)
plt.show()


In [ ]:
# =============================================================================
# 1️⃣3️⃣ Modeli Google Drive'a Kaydet (Opsiyonel)
# =============================================================================

# Google Drive'ı bağla
from google.colab import drive
drive.mount('/content/drive')

# Sonuçları kaydet
import shutil
save_dir = '/content/drive/MyDrive/NAT_BreaKHis_Results/'
os.makedirs(save_dir, exist_ok=True)

shutil.copy('nat_best_model.pth', save_dir)
shutil.copy('training_history.png', save_dir)
shutil.copy('confusion_roc.png', save_dir)
shutil.copy('per_magnification.png', save_dir)

print(f"✅ All results saved to: {save_dir}")
print("\n📁 Files saved:")
for f in os.listdir(save_dir):
    print(f"   - {f}")


---
## ✅ Tamamlandı!

### 📊 Sonuçlar:
- `nat_best_model.pth` - Eğitilmiş NAT model
- `training_history.png` - Eğitim grafikleri (Loss, Accuracy, F1, AUC)
- `confusion_roc.png` - Confusion Matrix & ROC Curve
- `per_magnification.png` - 40X, 100X, 200X, 400X karşılaştırma

### 🚀 Kullanım:
```python
# Modeli yükle
model = nat_tiny(num_classes=2)
model.load_state_dict(torch.load('nat_best_model.pth'))
model.eval()

# Tahmin yap
output = model(image_tensor)
prediction = output.argmax(dim=1)
```
